# 10.3 Time Variability

Many X-ray astronomical sources are time variable. The SIMPUT format allows to characterize the source
variability in several ways:
   * As a (energy dependent) light curve, specified through an ASCII file (parameter LCFile, which expects a file in which each row contains two numbers, the time and the flux at that time). Make sure to end the ASCII file with a new line.  
   * As a stochastic process defined through its power spectrum, an ASCII file where each row contains a frequency and the power at that frequency,  
   * As parameters of the power spectrum, which is defined by the sum of several Lorentzians and a zero-centered low frequency QPO. 
   
In addition the SIMPUT format also allows the specification of source variability through energy dependent pulse profiles.  
The Time Variability extension can be used to describe the time dependence of the source flux but it can also be used to create a dynamic spectrum with the source i.e. spectral model of the source that varies with time. The TIMING extension for SIMPUT files can be used to define a spectrum at a given time or phase in a periodic source with the relative flux of the source for the respective spectrum. If this is done the SRC_CAT should not contain a ’NULL’ in the Spectrum column. Note that the timing extension can have any arbitrary name as long at is correctly referenced in the SIMPUT source catalogue. For simplicity, we will call it TIMING in the
following, which is the default.


In [ ]:
import matplotlib.colors as colors
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas
import time
import tempfile
from funcs import run_comm

from astropy.io import fits
from astropy.visualization import astropy_mpl_style
from datetime import datetime
from subprocess import check_call, STDOUT
from xspec import Xset, Plot, AllData, ModelManager, Spectrum, Model, AllModels, Fit

plt.style.use(astropy_mpl_style)


In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]
xmldir = f"{SIXTE}/share/sixte/instruments/athena-wfi/wfi_wo_filter_B4C"

## 10.3.1 Ligth curve

### `simputfile` call with the spectrum *mcrab.xcm* (same as the one created in *tutorial-general-intro.ipynb*) with ligth curve

In [ ]:
AllModels.clear()
base = "mcrab"
xcm = f"{base}.xcm"
# define XSPEC parameters
Xset.abund = "wilm"
Xset.cosmo = "70 0. 0.73"
Xset.xsect = "bcmc"
mcmod = Model("phabs*pegpwrlw")
mcmod.phabs.nH = 0.2
mcmod.pegpwrlw.PhoIndex = 2.05
mcmod.pegpwrlw.eMin = 2.
mcmod.pegpwrlw.eMax = 10.
mcmod.pegpwrlw.norm = 21.6
mcmod.show()
AllModels.calcFlux("2. 10.")
simput_flux = AllModels(1).flux[0]

In [ ]:
clobber = True
# If clobber is true and the file exists, it will be removed before creating a new one.
if os.path.exists(xcm):
    if clobber:
        os.remove(xcm)
    else:
        raise Exception(f"Xspec file ({xcm}) already exists: it will not be overwritten") 
Xset.save(xcm)

In [ ]:
#download example light curve from:  
#      https://www.sternwarte.uni-erlangen.de/research/sixte/downloads/example_lightcurve.dat 

simput_lightcurve = f'{base}_lightcurve.fits'
# Simputfile with the lightcurve
comm = (f"simputfile Simput={simput_lightcurve} Src_Name=first RA=0.0  Dec=0.0 srcFlux={simput_flux} " + 
        f"Elow=0.1 Eup=15 NBins=1000 logEgrid=yes Emin=2 Emax=10 MJDREF=55000 " +
        f"LCFile=example_lightcurve.dat XSPECFile={xcm} clobber=True")
#print(comm)
run_comm(comm, "Creating simputfile with light curve")


### Inspect *mcrab_lightcurve.fits*

In [ ]:
f = fits.open(simput_lightcurve)
f.info()
f["TIMING"].columns.info()
f.close()

### <span style="color:blue">$\textbf{Exercise}$</span>:
#### Define a 1000 second light curve in a text file, linearly decreasing in flux to 0. Then create a SIMPUT file and simulate it again with `runsixt`.

In [ ]:
# 1000s light curve in a text file
lcurve = "lc_tozero.txt"
t = np.linspace(0, 1000, 1000)
flx = np.linspace(1, 0, 1000)
a = np.array([t, flx])
np.savetxt(lcurve, a.T)

In [ ]:
#create simputfile
simputfile = "mcrab_lightcurve_tozero.fits"
comm = (f"simputfile Simput={simputfile} Src_Name=first RA=0. Dec=0. srcFlux={simput_flux} Elow=0.01 Eup=20. " +
        f"NBins=1000 logEgrid=yes Emin=2. Emax=10. XSPECFile={xcm} clobber=yes MJDREF=55000 LCFile={lcurve}")
#print(comm)
run_comm(comm, 'Creating simput file')

In [ ]:
# run simulation with simputfile from lightcurve
evtfile = "sim_mcrab_lightcurve_tozero.fits"
xml = "{}/ld_wfi_ff_large.xml".format(xmldir)
comm = (f"runsixt XMLFile={xml} RA=0. Dec=0. Prefix='' Simput={simputfile} EvtFile={evtfile} " +
        f"MJDREF=55000 Exposure=1000 clobber=yes")
#print(comm)
run_comm(comm, "Running simulation of lightcurve data")

### <span style="color:blue">$\textbf{Exercise}$</span>:
#### Create a light curve from the event file and verify it decreases linearly to 0.

In [ ]:
# create light curve & verify that it decreases to zero
lcurve = "mcrab.lc"
comm = (f"makelc EvtFile={evtfile} Lightcurve={lcurve} length=1000.0 dt=1.0")
#print(comm)
run_comm(comm, "Creating lightcurve")  

In [ ]:
# convert light curve to [TIME, COUNTS] to plot
f=fits.open(lcurve)
TIMEDEL = f["COUNTS"].header["TIMEDEL"]
TSTART = f["COUNTS"].header["TSTART"]
data = f["COUNTS"].data
counts = data["COUNTS"]
times = np.arange(len(counts))*TIMEDEL + TSTART
times_middle = times + 0.5 * TIMEDEL
err_counts = np.sqrt(counts)
f.close()
# ALSO usinf 'fdump'
#comm = (f"fdump infile='{lcurve}[COUNTS][col TIME=#row*TIMEDEL, COUNTS]' clobber=yes outfile=check_lc.txt col='-' row='-' " +
#        f"prhead=no showrow=no showunit=no")
#run_comm(comm, "Converting lc to TIME, COUNTS")
#datalc = pandas.read_table("check_lc.txt", skiprows=0, sep="\s+")
#times = datalc.TIME
#counts = datalc.COUNTS

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,3))
ax.errorbar(times_middle, counts, yerr=err_counts, fmt='.',alpha=0.5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Counts");

### <span style="color:blue">$\textbf{Exercise}$</span>:
#### Extract the light curve again, but this time with a bin size of 2 s. Plot the light curve as a count rate

In [ ]:
# extract lc again but now with a bin size of 2s . Plot counts/binsize
lcurve = "mcrab_bs2.lc"
# Only changes dt ---> dt=2.0
comm = (f"makelc EvtFile={evtfile} Lightcurve={lcurve} length=1000.0 dt=2.0")
#print(comm)
run_comm(comm, "Creating lightcurve")

In [ ]:
# convert light curve to TIME COUNTS to plot
# With FTOOL fdump:
#comm = (f"fdump infile='{lcurve}[COUNTS][col TIME=#row*TIMEDEL, RATE=COUNTS/TIMEDEL]' clobber=yes " +
#        f"outfile=check_lc_bs2.txt col='-' row='-' prhead=no showrow=no showunit=no")
#run_comm(comm, "Converting lc to TIME, RATE")
#datalc = pandas.read_table("check_lc_bs2.txt", skiprows=0, sep="\s+")
#times = datalc.TIME
#rates = datalc.RATE

# With Astropy:
f=fits.open(lcurve)
TIMEDEL = f["COUNTS"].header["TIMEDEL"]
TSTART = f["COUNTS"].header["TSTART"]
data = f["COUNTS"].data
counts = data["COUNTS"]
times = np.arange(len(counts))*TIMEDEL + TSTART
rates = counts/TIMEDEL
times_middle = times + 0.5 * TIMEDEL
err_rates = np.sqrt(counts)/TIMEDEL
f.close()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,3))
ax.errorbar(times_middle, rates, yerr=err_rates, fmt='.', alpha=0.5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Rate (ct/s)");


## 10.3.2 Periodic variability

### <span style="color:blue">$\textbf{Exercise}$</span>:
#### Download the above SIMPUT of the Crab (https://www.sternwarte.uni-erlangen.de/research/sixte/simput/crab.simput.tgz) and compare it with the time variable SIMPUT you created in Sect. 10.3.1. Note that the light curve extension in the above file you just downloaded has a different name (which?). Can you spot all differences in the light curve extension between these two SIMPUT files?


In [ ]:
f_lc = fits.open("mcrab_lightcurve_tozero.fits")
f_pv = fits.open('crab.simput')
print('Time variable SIMPUT from 10.3.1: ')
f_lc.info()
print('Downloaded SIMPUT of the Crab: ')
f_pv.info()

print('Differences between both in the "TIMING" extension')
print(f_lc[3].columns.names)
print(f_pv[3].columns.names)

## 10.3.3 Power Spectrum

#### Add a QPO with a high Q-factor at a frequency of 0.01 Hz to the script that generated *mcrab.fits*. Then rerun the simulation and extract a light curve again.

In [ ]:
# recreate simput file
simput_lightcurve_pwr = f'{base}_lightcurve_pwr.fits'
# QPO added with Q1f, Q1Q, Q1rms
comm = (f"simputfile Simput={simput_lightcurve_pwr} Src_Name=first RA=0.0  Dec=0.0 srcFlux={simput_flux} Elow=0.01 Eup=20. " +
        f"NBins=1000 logEgrid=yes Emin=2 Emax=10 MJDREF=55000 XSPECFile=mcrab.xcm Q1f=0.01 Q1Q=180 Q1rms=0.01 clobber=True")
#print(comm)
run_comm(comm, "Creating simputfile with power spectrum")

In [ ]:
#run simulation with simputfile from lightcurve
evtfile = f"sim_evt_{base}_lightcurve_pwr.fits"
xml = f"{xmldir}/ld_wfi_ff_large.xml"
comm = (f"runsixt XMLFile={xml} RA=0. Dec=0. Prefix='' Simput={simput_lightcurve_pwr} " + 
        f"MJDREF=55000 EvtFile={evtfile} Exposure=1000 clobber=yes")
#print(comm)
run_comm(comm, "Running simulation of lightcurve data")

In [ ]:
# Create lightcurve
lcurve = f"sim_{base}_pwr.lc"
comm = (f"makelc EvtFile={evtfile} Lightcurve={lcurve} length=1000.0 dt=1.0")
#print(comm)
run_comm(comm, "Creating lightcurve")

In [ ]:
# Plot
f=fits.open(lcurve)
TIMEDEL = f["COUNTS"].header["TIMEDEL"]
TSTART = f["COUNTS"].header["TSTART"]
data = f["COUNTS"].data
counts = data["COUNTS"]
times = np.arange(len(counts))*TIMEDEL + TSTART
times_middle = times + 0.5 * TIMEDEL
err_counts = np.sqrt(counts)
f.close()
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,6))
ax.errorbar(times_middle, counts, yerr=err_counts, fmt='.',alpha=0.5)
fig.suptitle("QPO ligth curve")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Counts");

### <span style="color:blue">$\textbf{Exercise}$</span>:
#### Use the FITS row filter options (`fhelp` `rowfilter`) to select only events that belong to our source and that are in the energy range between 0.5 keV and 1.0 keV.

In [ ]:
evtfile = f"sim_evt_{base}_lightcurve_pwr.fits"
lcurve_filt = f"sim_{base}_pwr_filt.lc"
comm = (f"makelc EvtFile=\"{evtfile}[EVENTS][(SIGNAL>=0.5) && (SIGNAL<=1.) && (SRC_ID[1]==1)]\" Lightcurve={lcurve_filt} " +
        f"length=1000. dt=1. clobber=yes")
#print(comm)
run_comm(comm, "Creating lightcurve")

In [ ]:
# Plot
f=fits.open(lcurve_filt)
TIMEDEL = f["COUNTS"].header["TIMEDEL"]
TSTART = f["COUNTS"].header["TSTART"]
data = f["COUNTS"].data
counts = data["COUNTS"]
err_counts = np.sqrt(counts)
times = np.arange(len(counts))*TIMEDEL + TSTART
times_middle = times + 0.5 * TIMEDEL
f.close()
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,6))
fig.suptitle("Filtered ligth curve")
ax.errorbar(times_middle, counts, yerr=err_counts, fmt='.',alpha=0.5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Counts");